In [14]:
import pandas as pd
import json
import numpy as np

In [15]:
from censusAPI import myAPI
from geo import stco

In [16]:
#data = 'P001001'
#years = ['2000','2010']

In [17]:
def get_data(year,data):
    cols = f'NAME,{data}'
    data_cols = ['id','NAME',f'{data}','yr']
    df = pd.DataFrame()
    base_url = f'https://api.census.gov/data/{year}/dec/sf1'
    
    for st,co in stco.items():
        for c in co:
            sub_url = f'{base_url}?get={cols}&for=county%20subdivision:*&in=state:{st}&in=county:{c}&key={myAPI}'
            dff = pd.read_json(sub_url)
            dff.columns = dff.iloc[0]
            dff = dff[1:]
            dff['yr'] = year
            df = pd.concat([df,dff])

    df['id']=df.state+df.county+df['county subdivision']
    df['id']=df['id'].astype(int)
    
    pl_url = f'{base_url}?get={cols}&for=place:*&in=state:36&key={myAPI}'
    nyp = pd.read_json(pl_url)
    nyp.columns = nyp.iloc[0]
    nyp = nyp[1:]
    nyp['yr'] = year
    nyp['id']=nyp.state+nyp.place
    nyp['id']=nyp['id'].astype(int)
    
    df,nyp=df[data_cols],nyp[data_cols]
    subpl = pd.concat([df,nyp])
    return subpl

In [18]:
subpl00 = get_data('2000','P001001')
subpl10 = get_data('2010','P001001')

In [19]:
geo00 = pd.read_csv('../data/geo/subpl00.csv')
geo10 = pd.read_csv('../data/geo/subpl10.csv')

In [20]:
subpl00 = pd.merge(geo00,subpl00,how='left',left_on='id_00',right_on='id')
subpl10 = pd.merge(geo10,subpl10,how='left',left_on='id_10',right_on='id')

In [21]:
data_cols = ['stco','geoid','name','P001001','yr']
subpl00,subpl10 = subpl00[data_cols],subpl10[data_cols]
subpl = pd.concat([subpl00,subpl10])
subpl['P001001']=subpl['P001001'].astype(int)

In [22]:
subpl = pd.pivot_table(subpl,values='P001001',index=['stco','geoid','name'],columns='yr',\
                      aggfunc=np.sum,fill_value=0,margins=False).reset_index()

In [23]:
subpl.head()

yr,stco,geoid,name,2000,2010
0,9001,900100000,"County subdivisions not defined, Fairfield Cou...",0,0
1,9001,900104720,"Bethel town, Fairfield County, Connecticut",18067,18584
2,9001,900108070,"Bridgeport town, Fairfield County, Connecticut",139529,144229
3,9001,900108980,"Brookfield town, Fairfield County, Connecticut",15664,16452
4,9001,900118500,"Danbury town, Fairfield County, Connecticut",74848,80893


### Total Housing Units

In [24]:
#cols = f'NAME,H001001'
subpl00 = get_data('2000','H001001')
subpl10 = get_data('2010','H001001')
subpl00 = pd.merge(geo00,subpl00,how='left',left_on='id_00',right_on='id')
subpl10 = pd.merge(geo10,subpl10,how='left',left_on='id_10',right_on='id')
data_cols = ['stco','geoid','name','H001001','yr']
subpl00,subpl10 = subpl00[data_cols],subpl10[data_cols]
subpl_h = pd.concat([subpl00,subpl10])
subpl_h['H001001']=subpl_h['H001001'].astype(int)
subpl_h = pd.pivot_table(subpl_h,values='H001001',index=['stco','geoid','name'],columns='yr',\
                      aggfunc=np.sum,fill_value=0,margins=False).reset_index()

In [25]:
subpl_h.head()

yr,stco,geoid,name,2000,2010
0,9001,900100000,"County subdivisions not defined, Fairfield Cou...",0,0
1,9001,900104720,"Bethel town, Fairfield County, Connecticut",6653,7310
2,9001,900108070,"Bridgeport town, Fairfield County, Connecticut",54367,57012
3,9001,900108980,"Brookfield town, Fairfield County, Connecticut",5781,6562
4,9001,900118500,"Danbury town, Fairfield County, Connecticut",28519,31154


## Export tables to Excel

In [26]:
subpl.to_csv('output/subpl_pop_00-10.csv')
subpl_h.to_csv('output/subpl_hou_00-10.csv')

## Housing unit occupancy & vacancy

In [28]:
#cols = f'NAME,H001001'
subpl00 = get_data('2000','H003002')
subpl10 = get_data('2010','H003002')
subpl00 = pd.merge(geo00,subpl00,how='left',left_on='id_00',right_on='id')
subpl10 = pd.merge(geo10,subpl10,how='left',left_on='id_10',right_on='id')
data_cols = ['stco','geoid','name','H003002','yr']
subpl00,subpl10 = subpl00[data_cols],subpl10[data_cols]
subpl_h_o = pd.concat([subpl00,subpl10])
subpl_h_o['H003002']=subpl_h_o['H003002'].astype(int)
subpl_h_o = pd.pivot_table(subpl_h_o,values='H003002',index=['stco','geoid','name'],columns='yr',\
                      aggfunc=np.sum,fill_value=0,margins=False).reset_index()

In [29]:
#cols = f'NAME,H001003'
subpl00 = get_data('2000','H003003')
subpl10 = get_data('2010','H003003')
subpl00 = pd.merge(geo00,subpl00,how='left',left_on='id_00',right_on='id')
subpl10 = pd.merge(geo10,subpl10,how='left',left_on='id_10',right_on='id')
data_cols = ['stco','geoid','name','H003003','yr']
subpl00,subpl10 = subpl00[data_cols],subpl10[data_cols]
subpl_h_v = pd.concat([subpl00,subpl10])
subpl_h_v['H003003']=subpl_h_v['H003003'].astype(int)
subpl_h_v = pd.pivot_table(subpl_h_v,values='H003003',index=['stco','geoid','name'],columns='yr',\
                      aggfunc=np.sum,fill_value=0,margins=False).reset_index()

In [30]:
subpl_h_o.to_csv('output/subpl_hou_o_00-10.csv')
subpl_h_v.to_csv('output/subpl_hou_v_00-10.csv')